In [10]:
from mosa import Anneal
from dataset.reader import CTTFileReader
from dataset.converter import CTTFileConverter
from utill.optimization import *
from utill import *
from utill.variables import *

import pandas as pd
import numpy as np

Парсим данные из .ctt файла

In [11]:
# Parse data from ctt file
file_reader = CTTFileReader(FILE_PATH)
ctt_data = file_reader.get_data()
ctt_data

{'courses': {'c0001': {'teacher_id': 't000',
   'lectures': 6,
   'min_days': 4,
   'students': 130},
  'c0002': {'teacher_id': 't001',
   'lectures': 6,
   'min_days': 4,
   'students': 75},
  'c0004': {'teacher_id': 't002',
   'lectures': 7,
   'min_days': 3,
   'students': 117},
  'c0005': {'teacher_id': 't003',
   'lectures': 3,
   'min_days': 3,
   'students': 75},
  'c0014': {'teacher_id': 't004',
   'lectures': 1,
   'min_days': 1,
   'students': 65},
  'c0015': {'teacher_id': 't005',
   'lectures': 8,
   'min_days': 3,
   'students': 65},
  'c0016': {'teacher_id': 't006',
   'lectures': 7,
   'min_days': 3,
   'students': 65},
  'c0017': {'teacher_id': 't007',
   'lectures': 2,
   'min_days': 2,
   'students': 65},
  'c0024': {'teacher_id': 't008',
   'lectures': 4,
   'min_days': 3,
   'students': 55},
  'c0025': {'teacher_id': 't009',
   'lectures': 8,
   'min_days': 3,
   'students': 55},
  'c0078': {'teacher_id': 't010',
   'lectures': 5,
   'min_days': 4,
   'students': 55

Генерируем популяцию

In [12]:

file_converter = CTTFileConverter(ctt_data)
population = file_converter.get_set_of_solutions()

In [13]:
# Prepare algorithm
opt = Anneal()

opt.verbose=True
opt.population = population

# Assuming 'opt' is an instance of the Anneal class from mosa.py
opt.initial_temperature = 10.0
opt.temperature_decrease_factor = 0.95
opt.number_of_temperatures = 100
opt.number_of_iterations = 7000
opt.archive_size = 10000
opt.maximum_archive_rejections=1000

opt.mc_step_size={course_id: 4 for course_id in population.keys()}

# Assuming 'population' is your set of courses
opt.swap_move = {course_id: 7.0 for course_id in population.keys()}

opt.no_repeated_elements = {population_key: True for population_key in population.keys()}
opt.sort_solution_elements = {population_key: True for population_key in population.keys()}

opt.change_value_move = {population_key: 7.0 for population_key in population.keys()}

# Set maximum number of solution elements based on the maximum lectures for each course
opt.number_of_solution_elements = {population_key: ctt_data['courses'][population_key]['lectures'] for population_key in population.keys()}


--------------------------------------------------
    MULTI-OBJECTIVE SIMULATED ANNEALING (MOSA)    
--------------------------------------------------
         Developed by Prof. Roberto Gomes         
   Universidade Federal do ABC (UFABC), Brazil    




In [14]:
def fobj(solution):
    penalty = calculate_hard_constraints_penalty(solution, ctt_data)
    # robustness = calculate_robustness(solution, ctt_data, disruption_scenarios)
    # return penalty, robustness
    print(penalty)
    return penalty,

In [15]:
%%time
opt.evolve(fobj)

--- BEGIN: Evolving a solution ---

Looking for a solution in the checkpoint file...
Done!
Trying to load the archive from file archive.json...
Done!
------
Keys in the population/solution dictionaries:
    ['c0001']:
        Number of elements in the solution: 6
        Discrete sample space
        Number of elements in the population: 24
        Elements cannot be repeated in the solution: True
        Selection weight of this key: 1.000000
        Weight of 'change value' trial move: 7.000000
        Weight of 'swap' trial move: 7.000000
        Solution sorted after trial move: False
        Elements selected at random from the population
    ['c0002']:
        Number of elements in the solution: 6
        Discrete sample space
        Number of elements in the population: 54
        Elements cannot be repeated in the solution: True
        Selection weight of this key: 1.000000
        Weight of 'change value' trial move: 7.000000
        Weight of 'swap' trial move: 7.000000
   

In [16]:
opt.printx()

===
Solutions:
1) {'c0001': [[0, 4, 'B'], [3, 4, 'B'], [3, 1, 'B'], [3, 5, 'B'], [0, 0, 'B'], [1, 5, 'B']], 'c0002': [[2, 0, 'B'], [1, 4, 'B'], [0, 5, 'B'], [3, 0, 'C'], [1, 0, 'C'], [4, 2, 'B']], 'c0004': [[4, 0, 'B'], [3, 3, 'B'], [4, 3, 'B'], [2, 5, 'B'], [2, 3, 'B'], [3, 2, 'B'], [0, 2, 'B']], 'c0005': [[0, 3, 'B'], [1, 2, 'C'], [4, 1, 'C']], 'c0014': [3, 5, 'C'], 'c0015': [[0, 5, 'C'], [3, 3, 'B'], [2, 0, 'B'], [4, 2, 'C'], [3, 2, 'B'], [4, 3, 'B'], [3, 4, 'C'], [1, 0, 'B']], 'c0016': [[3, 1, 'C'], [4, 0, 'B'], [3, 0, 'C'], [2, 3, 'C'], [2, 4, 'C'], [0, 1, 'C'], [1, 4, 'B']], 'c0017': [[1, 2, 'C'], [4, 1, 'B']], 'c0024': [[4, 0, 'B'], [1, 4, 'C'], [2, 0, 'C'], [0, 3, 'B']], 'c0025': [[2, 3, 'B'], [2, 2, 'C'], [4, 1, 'C'], [0, 5, 'B'], [1, 2, 'B'], [3, 0, 'C'], [2, 5, 'B'], [1, 3, 'C']], 'c0078': [[1, 0, 'B'], [3, 2, 'C'], [2, 4, 'B'], [0, 1, 'B'], [4, 5, 'C']], 'c0030': [[3, 3, 'S'], [4, 5, 'C'], [1, 5, 'F'], [2, 4, 'B'], [0, 4, 'S']], 'c0031': [[3, 3, 'G'], [4, 5, 'C'], [3, 2, 'C

In [17]:
opt.printstats()

In [18]:
opt.plotfront()

MOSAError: Index out of range!

In [ ]:
%%time
pruned=opt.prunedominated()
opt.printx(pruned)

In [ ]:
opt.printstats(pruned)

In [ ]:
opt.plotfront(pruned)

In [ ]:
trimmed=opt.trimx(xset=pruned,thresholds=[None,100])
opt.printx(trimmed)

In [ ]:
sol=opt.reducex(xset=trimmed,index=0,nel=100)
opt.printx(sol)